#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수 오버라이딩
    * _ _ init _ _ () : 모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

In [264]:
## 모듈로딩
import torch                               # 텐서 관련 모듈
import torch.nn as nn                      # 인공신경망 관련 모듈
import torch.nn.functional as F            # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화 함수 등등)
import torch.optim as optim                # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary              # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *      # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *  # 분류 성능 지표 관련 모듈

In [265]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망 클래스 <hr>
    * 입력층 - 입력 : 피쳐수 고정
    * 출력층 - 출력 : 타겟수 고정
    * 은닉층 - 고정

In [266]:
# 모델 설계
# 데이터셋 : 피쳐 4개, 타겟 1개, 회귀
# 입 력 층 : 입력    4개    출력  20개    AF  ReLU
# 은 닉 층 : 입력   20개    출력  100개   AF  ReLU
# 출 력 층 : 입력  100개    출력  1개     AF  X , (Sigmoid & Softmax) 분류 일 때 사용

In [267]:
class MyModel(nn.Module):
    
    # 인스턴스 / 객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func)
    def __init__(self):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(3, 20)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20, 100)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100, 1)      # W 100 + b 1 => 1P, 101 * 1 = 101개 변수


    # 순방향 / 전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4 + b
        y = F.relu(y)               # 0 <= y    ---->   죽은 relu ==> leakyReLu
        
        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x20W20 + b
        y = F.relu(y)               

        return self.output_layer(y)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x100W100 + b

In [268]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module):
    
    # 인스턴스 / 객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func)
    def __init__(self, in_features):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features, 20)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20, 100)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100, 1)      # W 100 + b 1 => 1P, 101 * 1 = 101개 변수


    # 순방향 / 전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4 + b
        y = F.relu(y)               # 0 <= y    ---->   죽은 relu ==> leakyReLu
        
        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x20W20 + b
        y = F.relu(y)               

        return self.output_layer(y)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x100W100 + b

In [269]:
# 입력 피쳐 수가 동적인 모델델
class MyModel3(nn.Module):
    
    # 인스턴스 / 객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func)
    def __init__(self, in_features, in_out, h_out):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features, in_out)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(in_out, h_out)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(h_out, 1)      # W 100 + b 1 => 1P, 101 * 1 = 101개 변수


    # 순방향 / 전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4 + b
        y = F.relu(y)               # 0 <= y    ---->   죽은 relu ==> leakyReLu
        
        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x20W20 + b
        y = F.relu(y)               

        return self.output_layer(y)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x100W100 + b

In [270]:
## 모델 인스턴스 생성
m1 = MyModel()

In [271]:
for m in m1.named_parameters():
    print(m)

('input_layer.weight', Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119],
        [ 0.2710, -0.5435,  0.3462],
        [-0.1188,  0.2937,  0.0803],
        [-0.0707,  0.1601,  0.0285],
        [ 0.2109, -0.2250, -0.0421],
        [-0.0520,  0.0837, -0.0023],
        [ 0.5047,  0.1797, -0.2150],
        [-0.3487, -0.0968, -0.2490],
        [-0.1850,  0.0276,  0.3442],
        [ 0.3138, -0.5644,  0.3579],
        [ 0.1613,  0.5476,  0.3811],
        [-0.5260, -0.5489, -0.2785],
        [ 0.5070, -0.0962,  0.2471],
        [-0.2683,  0.5665, -0.2443],
        [ 0.4330,  0.0068, -0.3042],
        [ 0.2968, -0.3065,  0.1698],
        [-0.1667, -0.0633, -0.5551],
        [-0.2753,  0.3133, -0.1403],
        [ 0.5751,  0.4628, -0.0270],
        [-0.3854,  0.3516,  0.1792]], requires_grad=True))
('input_layer.bias', Parameter containing:
tensor([-0.3732,  0.3750,  0.3505,  0.5120, -0.3236, -0.0950, -0.0112,  0.0843,
        -0.4382, -0.4097,  0.3141, -0.1354,  0.2820,  0.0329,  0.1896,

In [272]:
# 학습 진행 ==> 모델 인스턴스 명 (데이터)
# 임의의 데이터
dataTS = torch.FloatTensor([[1, 3, 5], [2, 4, 6]])
targetTS = torch.FloatTensor([[4], [5]])

# 학습
pre_y = m1(dataTS)

print(pre_y)

calling forward()
tensor([[-0.1983],
        [-0.2537]], grad_fn=<AddmmBackward0>)


In [273]:
m3 = MyModel3(3, 100, 20)
pre_y = m3(dataTS)

print(pre_y)

calling forward()
tensor([[0.1149],
        [0.1381]], grad_fn=<AddmmBackward0>)


In [274]:
# 입력 피쳐 수가 동적인 모델델
class MyModel4(nn.Module):
    
    # 인스턴스 / 객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func)
    def __init__(self, in_features, in_out, h_out, h_cnt):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features, in_out)     # W 4 + b 1 => 1P, 5 * 20 = 100개 변수
        
        if h_cnt == 0:
            pass

        elif h_cnt == 1:
            self.hidden_layer = nn.Linear(in_out, h_out)  # W 20 + b 1 => 1P, 21 * 100 = 2100개 변수
        
        else:
            self.hidden_layer = nn.ModuleList()
            self.hidden_layer.append(nn.Linear(in_out, h_out))

        
            for i in range(h_cnt - 1):
                h_out2 = h_out + 20
                self.hidden_layer.append(nn.Linear(h_out, h_out2))
                h_out = h_out2

        self.output_layer = nn.Linear(h_out2, 1)      # W 100 + b 1 => 1P, 101 * 1 = 101개 변수


    # 순방향 / 전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4 + b
        y = F.relu(y)               # 0 <= y    ---->   죽은 relu ==> leakyReLu
        
        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x20W20 + b
        y = F.relu(y)               

        return self.output_layer(y)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + ~ + x100W100 + b

In [275]:
m4 = MyModel4(3, 20, 100, 5)

pre_y = m4(dataTS)
print(pre_y)

calling forward()


NotImplementedError: Module [ModuleList] is missing the required "forward" function